# RAG system that uses BigQuery to generate embeddings and perform a vector search.

In [1]:
# Import all the related libraries
from google.cloud import bigquery
from google import genai
from google.genai import types

In [2]:
# ---------------- Constants ----------------
PROJECT_ID = "qwiklabs-gcp-02-682b5eee4362"

# Latest stable Gemini model version

MODEL = "gemini-2.0-flash-001"

system_prompt = """
You are a helpful assistant specialized in answering questions related to Aurora Bay, Alaska.
Only respond based on the provided context. If the answer is not present in the context, reply with:
"I’m sorry, I don’t have enough information to answer that."
IMPORTANT: Do not use any external knowledge.
"""

searchable_query_prompt = """
You are a helpful assistant. Your task is to rewrite the user's natural language question into a concise and meaningful query suitable for vector search in BigQuery.
Focus on the core intent of the question. Remove unnecessary words, greetings, or filler phrases. Keep the output short, clear, and focused on keywords that best represent the user's information need without adding any additional content.
"""

In [3]:
# Define Big query client
bq_client = bigquery.Client(project=PROJECT_ID)

# Initialize the Gemini AI client to interact with the Vertex AI service.
# Parameters:
# - vertexai=True : specifies that the client should use Google Cloud Vertex AI as the backend.
# - project : GCP project ID where the Vertex AI resources are located.
# - location : Regional endpoint to connect to; "global" refers to a global endpoint.
gemini_client = genai.Client(
    vertexai=True,
    project=PROJECT_ID,
    location="global",
)

In [4]:
# Run the Vector search on Bigquery
def bq_vector_search(user_query):
  """
  Runs the vector search on bigquery and retrieves the context for LLM
  Parameter:
    user_query: Query provided by the user
  Returns:
    Context to be sent to the LLM to generate response
  """
  query = f"""
  SELECT
    query.query,
    base.content as question,
    base.answer
  FROM
      VECTOR_SEARCH(
          TABLE `AuroraBayFAQ.FAQ_embedded`,
          'ml_generate_embedding_result',
          (
              SELECT
                  ml_generate_embedding_result,
                  content AS query
              FROM
                  ML.GENERATE_EMBEDDING(
                      MODEL `AuroraBayFAQ.Embeddings`,
                      (SELECT '{user_query.strip()}' AS content)
                  )
          ),
          top_k => 5,
          options => '{{"fraction_lists_to_search": 0.01}}'
      );
  """

  job = bq_client.query(query)
  results = job.result()

  # Collect the results
  context_chunks = []
  for row in results:
      context_chunks.append(f"Question: {row.question}\nAnswer: {row.answer}")

  # Combine context for LLM input
  context = "\n\n".join(context_chunks)
  return context

In [5]:
def generate_searchable_query(user_input):
  """
  Generate a searchable query for Big query from User input
  Returns:
    Searchable query for Big query without any greeting, unnecessary fillers etc.,
  """
  response = gemini_client.models.generate_content(
    model = MODEL,
    contents = [f"Input Prompt: ${user_input}"],
    config = types.GenerateContentConfig(
        temperature = 1,
        top_p = 1,
        system_instruction=[types.Part.from_text(text=searchable_query_prompt)]
    )
  )
  return response.text


In [6]:
# Infinite loop to simulate the chat
while True:
  user_input = input("You: ")
  if user_input.strip().lower() in ["exit", "quit"]:
      print("Thank You!")
      break

  # Generate a searchable query from user's natural language query
  searchable_query = generate_searchable_query(user_input)

  # Get context from big query using user's query
  context = bq_vector_search(searchable_query)

  # Prompt with context to generate response to the user's query
  input_prompt = f"""
  Use the following context to answer the given question.
  Context: {context}
  Question: {user_input}
  """

  # Call LLM to get the response
  response = gemini_client.models.generate_content(
      model = MODEL,
      contents = [f"Input Prompt: ${input_prompt}"],
      config = types.GenerateContentConfig(
          temperature = 0,
          top_p = 1,
          system_instruction=[types.Part.from_text(text=system_prompt)]
      )
    )
  print(f"Gemini: {response.text}")

You: How is the weather out there in San Francisco Bay?
Gemini: I’m sorry, I don’t have enough information to answer that.
You: How is the weather out there in Aurora bay?
Gemini: Winters average between 10°F to 25°F, while summers are milder, around 50°F to 65°F. Temperatures can vary with coastal weather patterns.

You: How to reach Aurora bay?
Gemini: Most visitors arrive via regional flights into Aurora Bay Airport or by ferry from nearby coastal towns. Small cruise ships also make seasonal stops.

You: Is there any library available in Aurora bay?
Gemini: Yes. The Aurora Bay Public Library is located on Main Street, next to the town’s post office.

You: Exit
Thank You!
